In [ ]:
!curl -L "https://app.roboflow.com/ds/QnXSz7NNyH?key=hBj8suOmOZ" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
# Setup the necessary imports

import json
import pandas as pd
import numpy as np

In [ ]:
# Define the IoU Method

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
#Non-maxima supression for ensemble learning
import torch
import numpy as np

#from iou import intersection_over_union

def nms_ensemble(bboxes, iou_threshold):
    """
    Does Non Max Suppression given bboxes
    Parameters:
        bboxes (list): list of lists containing all bboxes with each bboxes
        specified as [x1, y1, x2, y2, class_pred, prob_score]
        iou_threshold (float): threshold where predicted bboxes is correct
        threshold (float): threshold to remove predicted bboxes (independent of IoU) 
        box_format (str): "midpoint" or "corners" used to specify bboxes
    Returns:
        list: bboxes after performing NMS given a specific IoU threshold
    """

    #Sort the bboxes per score, max score first.
    bboxes = sorted(bboxes, key=lambda x: x[5], reverse=True)
    bboxes_after_nms = []
    #Apply the nms algorithm and save them in bboxes_faster_nms
    while bboxes:
        chosen_box = bboxes.pop(0)
        
        bboxes = [
            box
            for box in bboxes
            if box[4] != chosen_box[4]
            or intersection_over_union(
                torch.tensor(chosen_box[:4]),
                torch.tensor(box[:4])
            )
            < iou_threshold
        ]
        bboxes_after_nms.append(chosen_box)
    
  
    return bboxes_after_nms

In [ ]:
#Intersection IOU
import torch


def intersection_over_union(boxes_preds, boxes_labels):
    """
    Calculates intersection over union
    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct Labels of Boxes (BATCH_SIZE, 4)
        box_format (str): (x1,y1,x2,y2)
    Returns:
        tensor: Intersection over union for all examples
    """
   
   #Preprare the bbox in the format (x1,y1,x2,y2)
    boxes_preds[2] = boxes_preds[2]+boxes_preds[0]
    boxes_preds[3] = boxes_preds[3]+boxes_preds[1]
    boxes_labels[2] = boxes_labels[2]+boxes_labels[0]
    boxes_labels[3] = boxes_labels[3]+boxes_labels[1]
    #Save each coordinate in variable to process later
    box1_x1 = boxes_preds[..., 0:1]
    box1_y1 = boxes_preds[..., 1:2]
    box1_x2 = boxes_preds[..., 2:3]
    box1_y2 = boxes_preds[..., 3:4]
    box2_x1 = boxes_labels[..., 0:1]
    box2_y1 = boxes_labels[..., 1:2]
    box2_x2 = boxes_labels[..., 2:3]
    box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # Need clamp(0) in case they do not intersect, then we want intersection to be 0
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

In [ ]:
# Locate and load the ground truth annotation
ground_truth_file = '/content/test/_annotations.coco.json'

with open(ground_truth_file) as f:
  gt_data = json.load(f)

In [ ]:
# Load the ground truth annotated data into a dataframe

gt_df = pd.DataFrame([], columns = ['Image_ID', 'FileName', 'Ground Truth','category_ID'])
for i in gt_data['images']:
    item = {"Image_ID": i['id'], "FileName" : i['file_name'], "Ground Truth" :0, "category_ID":0}
    gt_df = gt_df.append(item, ignore_index = True)
    
for dat in gt_data['annotations']:
    id = dat['image_id']
    gt_df.at[id,'Ground Truth'] = gt_df.at[id,'Ground Truth'] + 1
    gt_df.at[id,'category_ID'] = gt_df.at[id,'category_ID'] + 1

In [ ]:
gt_df

,Image_ID,FileName,Ground Truth,category_ID
0,0,India_002949_jpg.rf.f87f3b9211e824513f35c187f1...,1,1
1,1,Japan_004974_jpg.rf.f9ff48b8409ed568ada1998141...,2,2
2,2,Czech_000018_jpg.rf.f83399f36c7ec3b0c108138409...,0,0
3,3,India_002957_jpg.rf.f91453c4ba0110f9d7160fbd2b...,0,0
4,4,India_003048_jpg.rf.fcd792a612c2de1bf92cc8aaae...,1,1
...,...,...,...,...
774,774,India_003032_jpg.rf.4dc82421561026e85b5ac833b5...,0,0
775,775,Czech_000060_jpg.rf.4e8a627590c20f6fe8abdf779a...,0,0
776,776,Czech_000248_jpg.rf.4c0b69e100e7c1f9bc5a73292b...,2,2
777,777,Japan_005013_jpg.rf.42fc4257c84fac7cb4b9eb42bb...,3,3


In [ ]:
# Locate and load the predicted bounding boxes
# It is set up for ensemble learning, if you want to compute the results for FasterRCNN you can comment lines 3,6 and 7. If you want the results for Cascade RCNN you can comment lines 4,8 and 9.
bbox_pred_file_cascade = 'results.bbox.cascade.json'
bbox_pred_file_faster = 'results.bbox.faster.json'

with open(bbox_pred_file_cascade) as f:
  bbox_pred_data_cascade = json.load(f)
with open(bbox_pred_file_faster) as f:
  bbox_pred_data_faster = json.load(f)

In [ ]:
# Only choose predicted bounding boxes with certain threshold of score_thr
#Here happens the same as the cell before. Is set up for the ensemble learning. If you changed the cell for FasterRCNN here you have to comment lines 5,6,7. If you changed for CascadeRCNN then you hace to comment lines 8,9,10.
score_thr = 0.6
bbox_pred_threshold = list()
for pred in bbox_pred_data_cascade:
    if pred['score'] >= score_thr:
        bbox_pred_threshold.append(pred)
for pred in bbox_pred_data_faster:
    if pred['score'] >= score_thr:
        bbox_pred_threshold.append(pred)

In [ ]:
bbox_pred_threshold

[{'bbox': [149.89962768554688,
   584.272705078125,
   177.97982788085938,
   131.65362548828125],
  'category_id': 4,
  'image_id': 0,
  'score': 0.8770318627357483},
 {'bbox': [378.3721923828125,
   526.5234985351562,
   287.6337890625,
   189.43890380859375],
  'category_id': 4,
  'image_id': 0,
  'score': 0.7069708108901978},
 {'bbox': [403.4388732910156,
   537.2459716796875,
   84.811279296875,
   83.5091552734375],
  'category_id': 7,
  'image_id': 0,
  'score': 0.8161577582359314},
 {'bbox': [231.9942169189453,
   330.75885009765625,
   66.87843322753906,
   19.044342041015625],
  'category_id': 8,
  'image_id': 1,
  'score': 0.9959063529968262},
 {'bbox': [163.2940216064453,
   598.5093383789062,
   97.20527648925781,
   120.56036376953125],
  'category_id': 7,
  'image_id': 3,
  'score': 0.8314864039421082},
 {'bbox': [215.40235900878906,
   504.7322082519531,
   77.63471984863281,
   73.10940551757812],
  'category_id': 5,
  'image_id': 4,
  'score': 0.7873040437698364},
 {'

In [ ]:
# Get the predictions for each of the images

bbox_pred_count = [0] * len(gt_df)

for pred in bbox_pred_threshold:
    id = pred['image_id']
    bbox_pred_count[id] = bbox_pred_count[id] + 1


In [ ]:
# Add prediction column to the ground truth dataframe
gt_df['Prediction'] = bbox_pred_count

In [ ]:
gt_df

,Image_ID,FileName,Ground Truth,category_ID,Prediction
0,0,India_002949_jpg.rf.f87f3b9211e824513f35c187f1...,1,1,3
1,1,Japan_004974_jpg.rf.f9ff48b8409ed568ada1998141...,2,2,1
2,2,Czech_000018_jpg.rf.f83399f36c7ec3b0c108138409...,0,0,0
3,3,India_002957_jpg.rf.f91453c4ba0110f9d7160fbd2b...,0,0,1
4,4,India_003048_jpg.rf.fcd792a612c2de1bf92cc8aaae...,1,1,2
...,...,...,...,...,...
774,774,India_003032_jpg.rf.4dc82421561026e85b5ac833b5...,0,0,0
775,775,Czech_000060_jpg.rf.4e8a627590c20f6fe8abdf779a...,0,0,0
776,776,Czech_000248_jpg.rf.4c0b69e100e7c1f9bc5a73292b...,2,2,3
777,777,Japan_005013_jpg.rf.42fc4257c84fac7cb4b9eb42bb...,3,3,3


In [ ]:
# Determine the bounding box dimensions for the ground truth
gt_bbox_dimension = list()

for gt_image in gt_data['annotations']:
    item = {'image_id' : gt_image['image_id'], 'bbox' : gt_image['bbox'], 'category_ID': gt_image['category_id']}
    gt_bbox_dimension.append(item)

In [ ]:
# Determine the bounding box dimensions for the predicted boxes
pred_bbox_dimension = list()

for gt_image in bbox_pred_threshold:
    item = {'image_id' : gt_image['image_id'], 'bbox' : gt_image['bbox'], 'category_ID': gt_image['category_id'],'score': gt_image['score']}
    pred_bbox_dimension.append(item)

In [ ]:
# Convert the bounding boxes dimensions to a dataframe
pred_df_bbox_dimension = pd.DataFrame(pred_bbox_dimension)
gt_df_bbox_dimension = pd.DataFrame(gt_bbox_dimension)

In [ ]:
pred_df_bbox_dimension

,image_id,bbox,category_ID,score
0,0,"[149.89962768554688, 584.272705078125, 177.979...",4,0.877032
1,0,"[378.3721923828125, 526.5234985351562, 287.633...",4,0.706971
2,0,"[403.4388732910156, 537.2459716796875, 84.8112...",7,0.816158
3,1,"[231.9942169189453, 330.75885009765625, 66.878...",8,0.995906
4,3,"[163.2940216064453, 598.5093383789062, 97.2052...",7,0.831486
...,...,...,...,...
1214,776,"[7.811668395996094, 429.992431640625, 300.1302...",4,0.870254
1215,776,"[171.96200561523438, 464.50244140625, 149.5659...",4,0.690935
1216,777,"[239.8296661376953, 355.5826721191406, 281.231...",4,0.984861
1217,777,"[383.4734802246094, 526.3154296875, 89.4210205...",5,0.959916


In [ ]:
gt_df_bbox_dimension

,image_id,bbox,category_ID
0,0,"[342, 550, 370, 169]",4
1,1,"[230, 332, 68, 17]",8
2,1,"[268, 364, 14, 27]",1
3,4,"[222, 506, 71, 59]",5
4,5,"[213, 393, 382, 206]",4
...,...,...,...
984,776,"[187, 462, 121, 46]",4
985,776,"[2, 437, 168, 81]",4
986,777,"[381, 524, 84, 75]",5
987,777,"[267, 555, 31, 44]",5


In [ ]:
# Split into different columns for easy computation

gt_df_bbox_dimension[['x_min', 'y_min', 'x_inc', 'y_inc']] = pd.DataFrame(gt_df_bbox_dimension['bbox'].values.tolist())
pred_df_bbox_dimension[['x_min', 'y_min', 'x_inc', 'y_inc']] = pd.DataFrame(pred_df_bbox_dimension['bbox'].values.tolist())


In [ ]:
pred_df_bbox_dimension.sort_values('image_id')

,image_id,bbox,category_ID,score,x_min,y_min,x_inc,y_inc
0,0,"[149.89962768554688, 584.272705078125, 177.979...",4,0.877032,149.899628,584.272705,177.979828,131.653625
1,0,"[378.3721923828125, 526.5234985351562, 287.633...",4,0.706971,378.372192,526.523499,287.633789,189.438904
2,0,"[403.4388732910156, 537.2459716796875, 84.8112...",7,0.816158,403.438873,537.245972,84.811279,83.509155
3,1,"[231.9942169189453, 330.75885009765625, 66.878...",8,0.995906,231.994217,330.758850,66.878433,19.044342
4,3,"[163.2940216064453, 598.5093383789062, 97.2052...",7,0.831486,163.294022,598.509338,97.205276,120.560364
...,...,...,...,...,...,...,...,...
1213,776,"[155.10455322265625, 392.7109680175781, 75.059...",1,0.761256,155.104553,392.710968,75.059082,70.684937
1215,776,"[171.96200561523438, 464.50244140625, 149.5659...",4,0.690935,171.962006,464.502441,149.565948,54.059692
1217,777,"[383.4734802246094, 526.3154296875, 89.4210205...",5,0.959916,383.473480,526.315430,89.421021,71.955200
1216,777,"[239.8296661376953, 355.5826721191406, 281.231...",4,0.984861,239.829666,355.582672,281.231003,243.814911


In [ ]:
gt_df_bbox_dimension

,image_id,bbox,category_ID,x_min,y_min,x_inc,y_inc
0,0,"[342, 550, 370, 169]",4,342,550,370,169
1,1,"[230, 332, 68, 17]",8,230,332,68,17
2,1,"[268, 364, 14, 27]",1,268,364,14,27
3,4,"[222, 506, 71, 59]",5,222,506,71,59
4,5,"[213, 393, 382, 206]",4,213,393,382,206
...,...,...,...,...,...,...,...
984,776,"[187, 462, 121, 46]",4,187,462,121,46
985,776,"[2, 437, 168, 81]",4,2,437,168,81
986,777,"[381, 524, 84, 75]",5,381,524,84,75
987,777,"[267, 555, 31, 44]",5,267,555,31,44


In [ ]:
# Track the bounding boxes that have been matched to a ground truth or to a prediction
gt_df_bbox_dimension['selected'] = False
pred_df_bbox_dimension['selected'] = False

In [ ]:
pred_df_bbox_dimension

,image_id,bbox,category_ID,score,x_min,y_min,x_inc,y_inc,selected
0,0,"[149.89962768554688, 584.272705078125, 177.979...",4,0.877032,149.899628,584.272705,177.979828,131.653625,False
1,0,"[378.3721923828125, 526.5234985351562, 287.633...",4,0.706971,378.372192,526.523499,287.633789,189.438904,False
2,0,"[403.4388732910156, 537.2459716796875, 84.8112...",7,0.816158,403.438873,537.245972,84.811279,83.509155,False
3,1,"[231.9942169189453, 330.75885009765625, 66.878...",8,0.995906,231.994217,330.758850,66.878433,19.044342,False
4,3,"[163.2940216064453, 598.5093383789062, 97.2052...",7,0.831486,163.294022,598.509338,97.205276,120.560364,False
...,...,...,...,...,...,...,...,...,...
1214,776,"[7.811668395996094, 429.992431640625, 300.1302...",4,0.870254,7.811668,429.992432,300.130287,85.116455,False
1215,776,"[171.96200561523438, 464.50244140625, 149.5659...",4,0.690935,171.962006,464.502441,149.565948,54.059692,False
1216,777,"[239.8296661376953, 355.5826721191406, 281.231...",4,0.984861,239.829666,355.582672,281.231003,243.814911,False
1217,777,"[383.4734802246094, 526.3154296875, 89.4210205...",5,0.959916,383.473480,526.315430,89.421021,71.955200,False


In [ ]:
# A simple method to simplify getting bbox dimensions to help declutter the entire code
def get_bbox_dimension_simplified(item):
    return [item['x_min'], item['y_min'], float(item['x_min'] + item['x_inc']), float(item['y_min'] + item['y_inc'])]

In [ ]:
def get_bbox_dimension_simplified_V2(item):
    return [item['x_min'], item['y_min'], item['x_inc'] ,item['y_inc']]

In [ ]:
# Non - maxima suppression
IoU_Score = 0.5
image_ids = gt_df_bbox_dimension['image_id'].unique()
pred_bbox_dimension = list()

for image in image_ids:
    pred_items = pred_df_bbox_dimension.loc[pred_df_bbox_dimension['image_id'] == image]
    nms = list()
    for pred_index, pred_item in pred_items.iterrows():
      pred_bbox = get_bbox_dimension_simplified(pred_item)
      pred_cat = pred_item['category_ID']
      pred_bbox.append(pred_cat)
      pred_bbox.append(pred_item['score'])
      nms.append(pred_bbox)
      pred_bbox.append(pred_item['image_id'])
      nms.append(pred_bbox)
    nms = nms_ensemble(nms,0.5) 
    for gt_image in nms:
      item = {'image_id' : gt_image[6], 'bbox' : gt_image[:4], 'category_ID': gt_image[4],'score': gt_image[5]}
      pred_bbox_dimension.append(item)


In [ ]:
bbox_pred_count = [0] * len(gt_df)

for pred in pred_bbox_dimension:
    id = pred['image_id']
    bbox_pred_count[id] = bbox_pred_count[id] + 1

In [ ]:
# Add prediction column to the ground truth dataframe
gt_df['Prediction'] = bbox_pred_count

In [ ]:
gt_df

,Image_ID,FileName,Ground Truth,category_ID,Prediction
0,0,India_002949_jpg.rf.f87f3b9211e824513f35c187f1...,1,1,3
1,1,Japan_004974_jpg.rf.f9ff48b8409ed568ada1998141...,2,2,1
2,2,Czech_000018_jpg.rf.f83399f36c7ec3b0c108138409...,0,0,0
3,3,India_002957_jpg.rf.f91453c4ba0110f9d7160fbd2b...,0,0,0
4,4,India_003048_jpg.rf.fcd792a612c2de1bf92cc8aaae...,1,1,1
...,...,...,...,...,...
774,774,India_003032_jpg.rf.4dc82421561026e85b5ac833b5...,0,0,0
775,775,Czech_000060_jpg.rf.4e8a627590c20f6fe8abdf779a...,0,0,0
776,776,Czech_000248_jpg.rf.4c0b69e100e7c1f9bc5a73292b...,2,2,3
777,777,Japan_005013_jpg.rf.42fc4257c84fac7cb4b9eb42bb...,3,3,3


In [ ]:
pred_bbox_dimension = pd.DataFrame(pred_bbox_dimension)
pred_bbox_dimension

,image_id,bbox,category_ID,score
0,0,"[149.89962768554688, 584.272705078125, 327.879...",4,0.877032
1,0,"[403.4388732910156, 537.2459716796875, 488.250...",7,0.816158
2,0,"[378.3721923828125, 526.5234985351562, 666.005...",4,0.706971
3,1,"[231.9942169189453, 330.75885009765625, 298.87...",8,0.995906
4,4,"[215.40235900878906, 504.7322082519531, 293.03...",5,0.787304
...,...,...,...,...
976,776,"[155.10455322265625, 392.7109680175781, 230.16...",1,0.761256
977,776,"[171.96200561523438, 464.50244140625, 321.5279...",4,0.690935
978,777,"[239.8296661376953, 355.5826721191406, 521.060...",4,0.984861
979,777,"[383.4734802246094, 526.3154296875, 472.894500...",5,0.959916


In [ ]:
pred_bbox_dimension[['x_min', 'y_min', 'x_inc', 'y_inc']] = pd.DataFrame(pred_bbox_dimension['bbox'].values.tolist())
pred_bbox_dimension['selected'] = False

In [ ]:
pred_bbox_dimension

,image_id,bbox,category_ID,score,x_min,y_min,x_inc,y_inc,selected
0,0,"[149.89962768554688, 584.272705078125, 327.879...",4,0.877032,149.899628,584.272705,327.879456,715.926331,False
1,0,"[403.4388732910156, 537.2459716796875, 488.250...",7,0.816158,403.438873,537.245972,488.250153,620.755127,False
2,0,"[378.3721923828125, 526.5234985351562, 666.005...",4,0.706971,378.372192,526.523499,666.005981,715.962402,False
3,1,"[231.9942169189453, 330.75885009765625, 298.87...",8,0.995906,231.994217,330.758850,298.872650,349.803192,False
4,4,"[215.40235900878906, 504.7322082519531, 293.03...",5,0.787304,215.402359,504.732208,293.037079,577.841614,False
...,...,...,...,...,...,...,...,...,...
976,776,"[155.10455322265625, 392.7109680175781, 230.16...",1,0.761256,155.104553,392.710968,230.163635,463.395905,False
977,776,"[171.96200561523438, 464.50244140625, 321.5279...",4,0.690935,171.962006,464.502441,321.527954,518.562134,False
978,777,"[239.8296661376953, 355.5826721191406, 521.060...",4,0.984861,239.829666,355.582672,521.060669,599.397583,False
979,777,"[383.4734802246094, 526.3154296875, 472.894500...",5,0.959916,383.473480,526.315430,472.894501,598.270630,False


In [ ]:
image_ids = gt_df_bbox_dimension['image_id'].unique()
index_result = dict()
for image in image_ids:
    pred_items = pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image]
    gt_items = gt_df_bbox_dimension.loc[gt_df_bbox_dimension['image_id'] == image]
    found = False
    for pred_index, pred_item in pred_items.iterrows():
        # Get bbox dimensions
        pred_bbox = get_bbox_dimension_simplified_V2(pred_item)
        pred_cat = pred_item['category_ID']
        for gt_index, gt_item in gt_items.iterrows():
            
            # Check if ground truth bbox has already been matched with a prediction
            check_index = gt_item['selected']
            
            # If not matched, then try matching
            if check_index == False:
                gt_bbox = get_bbox_dimension_simplified(gt_item)
                gt_cat = gt_item['category_ID']
                # Compute IoU between ground truth and predicted bbox
                res = bb_intersection_over_union(pred_bbox, gt_bbox)
                
                # If it reaches the IoU_Score score
                if(res >= IoU_Score and ( gt_cat==pred_cat)):
                    found = True
                    
                    # Then such a prediction is a True Positive
#                    index_result[pred_index] = 'TP'
                    pred_bbox_dimension.at[pred_index,'selected'] = True
                    break

In [ ]:
gt_df_bbox_dimension

,image_id,bbox,category_ID,x_min,y_min,x_inc,y_inc,selected
0,0,"[342, 550, 370, 169]",4,342,550,370,169,False
1,1,"[230, 332, 68, 17]",8,230,332,68,17,False
2,1,"[268, 364, 14, 27]",1,268,364,14,27,False
3,4,"[222, 506, 71, 59]",5,222,506,71,59,False
4,5,"[213, 393, 382, 206]",4,213,393,382,206,False
...,...,...,...,...,...,...,...,...
984,776,"[187, 462, 121, 46]",4,187,462,121,46,False
985,776,"[2, 437, 168, 81]",4,2,437,168,81,False
986,777,"[381, 524, 84, 75]",5,381,524,84,75,False
987,777,"[267, 555, 31, 44]",5,267,555,31,44,False


In [ ]:
pred_bbox_dimension

,image_id,bbox,category_ID,score,x_min,y_min,x_inc,y_inc,selected
0,0,"[149.89962768554688, 584.272705078125, 327.879...",4,0.877032,149.899628,584.272705,327.879456,715.926331,False
1,0,"[403.4388732910156, 537.2459716796875, 488.250...",7,0.816158,403.438873,537.245972,488.250153,620.755127,False
2,0,"[378.3721923828125, 526.5234985351562, 666.005...",4,0.706971,378.372192,526.523499,666.005981,715.962402,True
3,1,"[231.9942169189453, 330.75885009765625, 298.87...",8,0.995906,231.994217,330.758850,298.872650,349.803192,True
4,4,"[215.40235900878906, 504.7322082519531, 293.03...",5,0.787304,215.402359,504.732208,293.037079,577.841614,True
...,...,...,...,...,...,...,...,...,...
976,776,"[155.10455322265625, 392.7109680175781, 230.16...",1,0.761256,155.104553,392.710968,230.163635,463.395905,False
977,776,"[171.96200561523438, 464.50244140625, 321.5279...",4,0.690935,171.962006,464.502441,321.527954,518.562134,True
978,777,"[239.8296661376953, 355.5826721191406, 521.060...",4,0.984861,239.829666,355.582672,521.060669,599.397583,True
979,777,"[383.4734802246094, 526.3154296875, 472.894500...",5,0.959916,383.473480,526.315430,472.894501,598.270630,True


In [ ]:
len(image_ids)

444

In [ ]:
tps = [0] * len(gt_df)
fps = [0] * len(gt_df)

# For each image, compute its True Positive and False Positive
for image in image_ids:
    tp = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['selected'] == True])
    fp = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['selected'] == False])
    
    tps[image] = tp
    fps[image] = fp


In [ ]:
print(len(tps))
print(len(fps))

779
779


In [ ]:
gt_df['True Positive'] = tps
gt_df['False Positive'] = fps

In [ ]:
# Compute False Negatives

x = np.array(gt_df['Ground Truth'] - gt_df['True Positive'])
y = np.zeros_like(x)
z = np.array([x, y])
z = np.amax(z, axis = 0)

gt_df['False Negative'] = z

In [ ]:
res = gt_df.loc[(gt_df['False Positive'] > 0) | (gt_df['False Negative'] > 0)]

In [ ]:
len(res)

265

In [ ]:
res

,Image_ID,FileName,Ground Truth,category_ID,Prediction,True Positive,False Positive,False Negative
0,0,India_002949_jpg.rf.f87f3b9211e824513f35c187f1...,1,1,3,1,2,0
1,1,Japan_004974_jpg.rf.f9ff48b8409ed568ada1998141...,2,2,1,1,0,1
5,5,Japan_004977_jpg.rf.fb015bff46cb976989085d5324...,3,3,3,2,1,1
6,6,Czech_000332_jpg.rf.fb175ade3df52646069f0ba717...,1,1,0,0,0,1
7,7,India_002956_jpg.rf.f69910539a78ed2d7692c259bd...,1,1,2,1,1,0
...,...,...,...,...,...,...,...,...
753,753,Japan_005015_jpg.rf.49b9387661db419bd58dd4254e...,3,3,2,2,0,1
762,762,India_003001_jpg.rf.47615322f90fc4dd25bddc049e...,1,1,3,1,2,0
766,766,Czech_000274_jpg.rf.4c8aef3e5580583063ea0c658e...,3,3,2,1,1,2
773,773,India_003051_jpg.rf.4f6346af8b13c1bb4ce707ddbf...,2,2,2,1,1,1


In [ ]:
TP = np.sum(gt_df['True Positive'])
FP = np.sum(gt_df['False Positive'])
FN = np.sum(gt_df['False Negative'])
recall = TP / (TP + FN); print("RECALL: ", recall)
precision = TP / (TP + FP); print("PRECISION: ", precision)
fscore = 2*recall*precision/(recall+precision); print("FSCORE: ", fscore)

RECALL:  0.7328629032258065
PRECISION:  0.7410805300713558
FSCORE:  0.7369488089204257


##FSCORE for multiple classes

In [ ]:
##FSCORE for multiple classes

tps_1 = [0] * len(gt_df)
fps_1 = [0] * len(gt_df)

tps_2 = [0] * len(gt_df)
fps_2 = [0] * len(gt_df)

tps_3 = [0] * len(gt_df)
fps_3 = [0] * len(gt_df)

tps_4 = [0] * len(gt_df)
fps_4 = [0] * len(gt_df)

tps_5 = [0] * len(gt_df)
fps_5 = [0] * len(gt_df)

tps_6 = [0] * len(gt_df)
fps_6 = [0] * len(gt_df)

tps_7 = [0] * len(gt_df)
fps_7 = [0] * len(gt_df)

In [ ]:
for image in image_ids:
    tps_1[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 1].loc[pred_bbox_dimension['selected'] == True])
    fps_1[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 1].loc[pred_bbox_dimension['selected'] == False])
    tps_2[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 2].loc[pred_bbox_dimension['selected'] == True])
    fps_2[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 2].loc[pred_bbox_dimension['selected'] == False])
    tps_3[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 3].loc[pred_bbox_dimension['selected'] == True])
    fps_3[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 3].loc[pred_bbox_dimension['selected'] == False])
    tps_4[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 4].loc[pred_bbox_dimension['selected'] == True])
    fps_4[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 4].loc[pred_bbox_dimension['selected'] == False])
    tps_5[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 5].loc[pred_bbox_dimension['selected'] == True])
    fps_5[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 5].loc[pred_bbox_dimension['selected'] == False])
    tps_6[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 6].loc[pred_bbox_dimension['selected'] == True])
    fps_6[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 6].loc[pred_bbox_dimension['selected'] == False])
    tps_7[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 7].loc[pred_bbox_dimension['selected'] == True])
    fps_7[image] = len(pred_bbox_dimension.loc[pred_bbox_dimension['image_id'] == image].loc[pred_bbox_dimension['category_ID']== 7].loc[pred_bbox_dimension['selected'] == False])

In [ ]:
gt_df['True Positive_1'] = tps_1
gt_df['False Positive_1'] = fps_1
gt_df['True Positive_2'] = tps_2
gt_df['False Positive_2'] = fps_2
gt_df['True Positive_3'] = tps_3
gt_df['False Positive_3'] = fps_3
gt_df['True Positive_4'] = tps_4
gt_df['False Positive_4'] = fps_4
gt_df['True Positive_5'] = tps_5
gt_df['False Positive_5'] = fps_5
gt_df['True Positive_6'] = tps_6
gt_df['False Positive_6'] = fps_6
gt_df['True Positive_7'] = tps_7
gt_df['False Positive_7'] = fps_7

In [ ]:
gt_df

,Image_ID,FileName,Ground Truth,category_ID,Prediction,True Positive,False Positive,False Negative,True Positive_1,False Positive_1,True Positive_2,False Positive_2,True Positive_3,False Positive_3,True Positive_4,False Positive_4,True Positive_5,False Positive_5,True Positive_6,False Positive_6,True Positive_7,False Positive_7
0,0,India_002949_jpg.rf.f87f3b9211e824513f35c187f1...,1,1,3,1,2,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1
1,1,Japan_004974_jpg.rf.f9ff48b8409ed568ada1998141...,2,2,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,Czech_000018_jpg.rf.f83399f36c7ec3b0c108138409...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,India_002957_jpg.rf.f91453c4ba0110f9d7160fbd2b...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,India_003048_jpg.rf.fcd792a612c2de1bf92cc8aaae...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,774,India_003032_jpg.rf.4dc82421561026e85b5ac833b5...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
775,775,Czech_000060_jpg.rf.4e8a627590c20f6fe8abdf779a...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
776,776,Czech_000248_jpg.rf.4c0b69e100e7c1f9bc5a73292b...,2,2,3,1,2,1,0,1,0,0,0,0,1,1,0,0,0,0,0,0
777,777,Japan_005013_jpg.rf.42fc4257c84fac7cb4b9eb42bb...,3,3,3,3,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0


In [ ]:
# Compute False Negatives

x = np.array(gt_df['Ground Truth'].loc[gt_df['category_ID']== 1] - gt_df['True Positive_1'])
y = np.zeros_like(x)
z = np.array([x, y])
z = np.amax(z, axis = 0)

gt_df['False Negative_1'] = z

x = np.array(gt_df['Ground Truth'].loc[gt_df['category_ID']== 2] - gt_df['True Positive_2'])
y = np.zeros_like(x)
z = np.array([x, y])
z = np.amax(z, axis = 0)

gt_df['False Negative_2'] = z

x = np.array(gt_df['Ground Truth'].loc[gt_df['category_ID']== 3] - gt_df['True Positive_3'])
y = np.zeros_like(x)
z = np.array([x, y])
z = np.amax(z, axis = 0)

gt_df['False Negative_3'] = z

x = np.array(gt_df['Ground Truth'].loc[gt_df['category_ID']== 4] - gt_df['True Positive_4'])
y = np.zeros_like(x)
z = np.array([x, y])
z = np.amax(z, axis = 0)

gt_df['False Negative_4'] = z

x = np.array(gt_df['Ground Truth'].loc[gt_df['category_ID']== 5] - gt_df['True Positive_5'])
y = np.zeros_like(x)
z = np.array([x, y])
z = np.amax(z, axis = 0)

gt_df['False Negative_5'] = z

x = np.array(gt_df['Ground Truth'].loc[gt_df['category_ID']== 6] - gt_df['True Positive_6'])
y = np.zeros_like(x)
z = np.array([x, y])
z = np.amax(z, axis = 0)

gt_df['False Negative_6'] = z

x = np.array(gt_df['Ground Truth'].loc[gt_df['category_ID']== 7] - gt_df['True Positive_7'])
y = np.zeros_like(x)
z = np.array([x, y])
z = np.amax(z, axis = 0)

gt_df['False Negative_7'] = z

In [ ]:
res1 = gt_df.loc[(gt_df['False Positive_1'] > 0) | (gt_df['False Negative_1'] > 0)]
res2 = gt_df.loc[(gt_df['False Positive_2'] > 0) | (gt_df['False Negative_2'] > 0)]
res3 = gt_df.loc[(gt_df['False Positive_3'] > 0) | (gt_df['False Negative_3'] > 0)]
res4 = gt_df.loc[(gt_df['False Positive_4'] > 0) | (gt_df['False Negative_4'] > 0)]
res5 = gt_df.loc[(gt_df['False Positive_5'] > 0) | (gt_df['False Negative_5'] > 0)]
res6 = gt_df.loc[(gt_df['False Positive_6'] > 0) | (gt_df['False Negative_6'] > 0)]
res7 = gt_df.loc[(gt_df['False Positive_7'] > 0) | (gt_df['False Negative_7'] > 0)]


In [ ]:
TP = np.sum(gt_df['True Positive_1'])
FP = np.sum(gt_df['False Positive_1'])
FN = np.sum(gt_df['False Negative_1'])
recall = TP / (TP + FN); print("RECALL_1: ", recall)
precision = TP / (TP + FP); print("PRECISION_1: ", precision)
fscore = 2*recall*precision/(recall+precision); print("FSCORE_1: ", fscore)

TP = np.sum(gt_df['True Positive_2'])
FP = np.sum(gt_df['False Positive_2'])
FN = np.sum(gt_df['False Negative_2'])
recall = TP / (TP + FN); print("RECALL_2: ", recall)
precision = TP / (TP + FP); print("PRECISION_2: ", precision)
fscore = 2*recall*precision/(recall+precision); print("FSCORE_2: ", fscore)

TP = np.sum(gt_df['True Positive_3'])
FP = np.sum(gt_df['False Positive_3'])
FN = np.sum(gt_df['False Negative_3'])
recall = TP / (TP + FN); print("RECALL_3: ", recall)
precision = TP / (TP + FP); print("PRECISION_3: ", precision)
fscore = 2*recall*precision/(recall+precision); print("FSCORE_3: ", fscore)

TP = np.sum(gt_df['True Positive_4'])
FP = np.sum(gt_df['False Positive_4'])
FN = np.sum(gt_df['False Negative_4'])
recall = TP / (TP + FN); print("RECALL_4: ", recall)
precision = TP / (TP + FP); print("PRECISION_4: ", precision)
fscore = 2*recall*precision/(recall+precision); print("FSCORE_4: ", fscore)

TP = np.sum(gt_df['True Positive_5'])
FP = np.sum(gt_df['False Positive_5'])
FN = np.sum(gt_df['False Negative_5'])
recall = TP / (TP + FN); print("RECALL_5: ", recall)
precision = TP / (TP + FP); print("PRECISION_5: ", precision)
fscore = 2*recall*precision/(recall+precision); print("FSCORE_5: ", fscore)

TP = np.sum(gt_df['True Positive_6'])
FP = np.sum(gt_df['False Positive_6'])
FN = np.sum(gt_df['False Negative_6'])
recall = TP / (TP + FN); print("RECALL_6: ", recall)
precision = TP / (TP + FP); print("PRECISION_6: ", precision)
fscore = 2*recall*precision/(recall+precision); print("FSCORE_6: ", fscore)

TP = np.sum(gt_df['True Positive_7'])
FP = np.sum(gt_df['False Positive_7'])
FN = np.sum(gt_df['False Negative_7'])
recall = TP / (TP + FN); print("RECALL_7: ", recall)
precision = TP / (TP + FP); print("PRECISION_7: ", precision)
fscore = 2*recall*precision/(recall+precision); print("FSCORE_7: ", fscore)


RECALL_1:  0.5636942675159236
PRECISION_1:  0.7283950617283951
FSCORE_1:  0.6355475763016157
RECALL_2:  0.0
PRECISION_2:  nan
FSCORE_2:  nan
RECALL_3:  0.25177304964539005
PRECISION_3:  0.5916666666666667
FSCORE_3:  0.35323383084577115
RECALL_4:  0.5912408759124088
PRECISION_4:  0.7751196172248804
FSCORE_4:  0.6708074534161491
RECALL_5:  0.6041666666666666
PRECISION_5:  0.7532467532467533
FSCORE_5:  0.6705202312138729
RECALL_6:  0.2727272727272727
PRECISION_6:  0.75
FSCORE_6:  0.39999999999999997
RECALL_7:  0.7549668874172185
PRECISION_7:  0.7651006711409396
FSCORE_7:  0.76


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':
